# DOC
## LAST UPDATE: 2022-02-16
## IPDC PROCEDURE
## STEP 3 (D): DECLUSTERING HEURISTIC

input: 

pickle files from step 0 
* "./data/pickle/H.gpickle"
* "./data/pickle/h.pickle"
* "./data/pickle/ebc.pickle" 

pickle file from step 1-2 - gaps df
* "./data/pickle/mygaps.pickle"

output:

pickle file of declustered gaps 
* "./data/pickle/gaps_declustered.pickle"

folium map of declustered gaps and csv file for modification (manual analysis)
* "./analysis/gaps_declustered_table.csv"
* "./analysis/gaps_declustered_plot.html"

# SETUP

In [2]:
# import packages
%run -i packages.py

# import parameters for plotting
%run -i parameters_plot.py

# IMPORT OBJECTS FROM PREVIOUS STEPS

H = nx.read_gpickle("./data/pickle/H.gpickle")
h = ig.Graph.Read_Pickle("./data/pickle/h.pickle")

ebc = pd.read_pickle("./data/pickle/ebc.pickle")
mygaps = pd.read_pickle("./data/pickle/mygaps.pickle")

# NX attribute dicts (for plotting)
ced = nx.get_edge_attributes(H, "coord") # coordinates of edges dictionary ced
ted = nx.get_edge_attributes(H, "category_edge") # type of edges dictionary ted
tnd = nx.get_node_attributes(H, "category_node") # type of nodes dictionary tnd
cnd = nx.get_node_attributes(H, "coord") # coordinates of nodes dictionary cnd

# make subfolder for storing output
os.mkdir("./analysis")

# RUN DECLUSTERING HEURISTIC

In [3]:
# set cut-off value for benefit metric, exclude gaps below
B_cutoff = 15300
mygaps = mygaps[mygaps["B"]>=B_cutoff].reset_index(drop = True)

# ADDING EBC VALUES / RANKING METRICS TO EACH EDGE in network "f" (which will be used for declustering)
f = h.copy()
for i in range(len(ebc)):
    f.es[i]["ebc"] = ebc.loc[i, "ebc_lambda"]
    

### make a subgraph of f that contains only overlapping gaps

my_edges = list(set([item for sublist in mygaps["path"] for item in sublist]))
c = f.copy()
c = c.subgraph_edges(edges = my_edges, 
                         delete_vertices=True)

cl = c.decompose() ### cl contains disconnected components 

gapranks = []
gapcoords = []

gapranks = []
gapcoords = []
gapedgeids = []

# looping through disconnected components
for comp in range(len(cl)):
    
    mc = cl[comp].copy() # mc: my component (current)

    #### decluster component:

    while len(mc.es) > 0:

        nodestack = [node.index for node in mc.vs() if mc.vs[node.index]["category_node"]=="multi" and mc.degree(node.index)!=2]
        nodecomb = [comb for comb in itertools.combinations(nodestack, 2)] ### all possible OD combis on that cluster
        sp = [] ### list of shortest paths between d1 multi nodes on that cluster:
        for mycomb in nodecomb:

            gsp = mc.get_shortest_paths(mycomb[0], 
                                                mycomb[1],
                                                weights = "length", 
                                                mode = "out", 
                                                output = "epath")[0]
            if gsp:
                sp.append(gsp)

        ### compute metrics for each path:
        if not sp:
            break

        lens = []
        cycs = []
        
        for p in sp:
            lens += [np.sum([mc.es[e]["length"] for e in p])]
            cycs += [np.sum([mc.es[e]["length"]*mc.es[e]["ebc"] for e in p])]
            
        norms = list(np.array(cycs)/np.array(lens))
        maxpath = sp[norms.index(max(norms))]
        gapranks.append(np.round(max(norms), 2))
        
        gapcoord_current = []
        edgeids_current = []
        
        
        for e in maxpath:
            edge_id = tuple(sorted(literal_eval(mc.es[e]["edge_id"])))
            edge_coords = [(c[1], c[0]) for c in ced[tuple(sorted(edge_id))].coords]
            gapcoord_current.append(edge_coords)
            edgeids_current.append(edge_id)
            
        mc.delete_edges(maxpath)
        gapcoords.append(gapcoord_current)
        gapedgeids.append(edgeids_current)


gap_dec = pd.DataFrame({"rank": gapranks, "coord": gapcoords, "id": gapedgeids})
gap_dec = gap_dec.sort_values(by = "rank", ascending = False).reset_index(drop = True)
gaps_before = len(gap_dec)
gap_dec = gap_dec[gap_dec["rank"]>=B_cutoff].copy().reset_index(drop = True)

### export to csv to start manual analysis
gap_dec["address"] = None
gap_dec["class"] = None
gap_dec["comments"] = None
gap_dec[["rank", "address", "class", "comments"]].to_csv(path_or_buf = "./analysis/gaps_declustered_table.csv", 
                                                         sep = ";", 
                                                         index = True, 
                                                         encoding = "utf-8", 
                                                         decimal = ",")

gap_dec.to_pickle("./data/pickle/gaps_declustered.pickle")

/var/folders/b0/lkvf88hn0673f5dlj9z0_2dr0000gn/T/ipykernel_13817/224840499.py:41: RuntimeWarning: Couldn't reach some vertices at src/paths/dijkstra.c:441
  gsp = mc.get_shortest_paths(mycomb[0],


# MAKE FOLIUM MAP OF DECLUSTERED GAPS

In [8]:
### SET UP MAP WITH FOLIUM

# make map object

copenhagen_coord = [55.6761, 12.5683]

m = folium.Map(location = copenhagen_coord, zoom_start = 13, tiles = None) 

# add tile layers to m
for key in ["osm", "st_lite_op3", "white_background"]:
    basemaps[key].add_to(m)
    
##### plot car, onlybike, and multinetworks #####
snw = folium.FeatureGroup(name = "All streets", show = True)
bnw = folium.FeatureGroup(name = "Bikeable", show = True) # bikeable edges 

sloc = []
bloc = []

for edge in H.edges:
    myloc = [(c[1], c[0]) for c in ced[tuple(sorted(edge))].coords]
    sloc.append(myloc)
    if ted[edge] != "car":
        bloc.append(myloc)
        
snw_line = folium.PolyLine(locations = sloc, weight = 2, color = "#dadada").add_to(snw)
bnw_line = folium.PolyLine(locations = bloc, weight = 3, color = "black").add_to(bnw)

snw.add_to(m)
bnw.add_to(m)


#### ADD TO MAP 


gaps_fg = folium.FeatureGroup("Declustered gaps", show = True)

for locs in gap_dec["coord"]:
    myline = folium.PolyLine(locations = locs,
                                weight = 4,
                                color = random.choice(["#ffa208", "#ff0000", "#ff55f7", "#9400d3", "#fae700"])).add_to(gaps_fg)

gaps_fg.add_to(m)


my_fg_nr = folium.FeatureGroup("Gap numbers", show = False)

# Add pop-ups with gap number

for i in range(len(gap_dec)):
    
    my_marker = folium.Marker(location = gap_dec.loc[i]["coord"][0][0], 
                                    popup = "Gap " + str(i)
                             ).add_to(my_fg_nr)
                  
my_fg_nr.add_to(m)

### ADD LAYER CONTROL AND SAVE / DISPLAY FOLIUM MAP
folium.LayerControl().add_to(m)  

m.save("./analysis/gaps_declustered_plot.html")

### END OF 02_D notebook